In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 1.22 MiB/s, done.
Resolving deltas: 100% (191/191), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.4 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla V100-SXM2-16GB!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import cudf
import cuml
import os
import numpy as np
import pandas as pd

In [ ]:
import re
import pandas as pd
import csv
import numpy as np
from collections import Counter
import nltk
import random
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import IsolationForest
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')
import gensim
from gensim.models import Word2Vec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the CSV file into a GPU DataFrame # Original data
#df2 = pd.read_csv('/content/drive/MyDrive/thesis/novaall.csv', sep=',')
df3 = pd.read_csv('/content/drive/MyDrive/thesis/nova-issue.csv', sep=',')
df4 = pd.read_csv('/content/drive/MyDrive/thesis/novaupdate.csv', sep=',')
df5 = pd.read_csv('/content/drive/MyDrive/thesis/nova_outlier.csv', sep=',')

In [ ]:
df3.shape

(3, 1)

In [ ]:
df4.shape

(10000, 1)

In [ ]:
df5.shape

(1000, 1)

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/thesis/novaall.csv', sep=',')

In [ ]:
samp = df1.head(30000)

In [ ]:
samp.head(5)

,message
0,2023-05-10 18:56:18.091 33 INFO nova.metadata....
1,2023-05-10 18:56:18.094 34 INFO nova.metadata....
2,2023-05-10 18:56:18.098 34 INFO nova.metadata....
3,2023-05-10 18:56:18.117 35 INFO nova.metadata....
4,2023-05-10 18:56:18.120 13 INFO nova.osapi_com...


In [ ]:
samp2 =df1.head(20000)

In [ ]:
df6 = pd.concat([samp, df5], axis=0)
df6.shape

(31000, 1)

In [ ]:
df7 = pd.concat([df6,samp2], axis = 0)

In [ ]:
df7.shape

(51000, 1)

In [ ]:
df8 = pd.concat([df7, df4], axis =0)

In [ ]:
df8.shape

(61000, 1)

In [ ]:
df9 = pd.concat([df8, df4], axis = 0)

In [ ]:
df9.shape

(71000, 1)

In [ ]:
df10 = pd.concat([df9, samp2], axis=0)

In [ ]:
df11 = pd.concat([df10, df5], axis=0)

In [ ]:
raw_out = pd.concat([df11, df4], axis=0)

In [ ]:
raw_out.to_csv('/content/drive/MyDrive/thesis/val/raw_val.csv', index=False)

In [ ]:
raw_out.shape

(102000, 1)

In [ ]:
rows_mtch = raw_out[raw_out['message'].str.contains(pattern, regex=True)]

In [ ]:
print(rows_mtch)

                                               message
83   2023-05-12 00:03:03.868 17 ERROR nova.api.open...
85   2023-05-12 00:03:11.247 18 ERROR nova.api.open...
104  2023-05-12 02:21:58.750 19 ERROR nova.api.open...
106  2023-05-12 02:22:02.731 14 ERROR nova.api.open...
109  2023-05-12 02:22:05.065 13 ERROR nova.api.open...
..                                                 ...
327  2023-05-12 05:02:58.476 11 ERROR nova.api.open...
329  2023-05-12 05:03:07.261 13 ERROR nova.api.open...
897  2023-05-13 10:55:57.092 11 ERROR nova.api.open...
899  2023-05-13 10:56:02.477 14 ERROR nova.api.open...
900  2023-05-13 10:56:02.488 14 ERROR nova.api.open...

[278 rows x 1 columns]


In [ ]:
df2.head(5)

,message
0,2023-05-10 18:56:18.091 33 INFO nova.metadata....
1,2023-05-10 18:56:18.094 34 INFO nova.metadata....
2,2023-05-10 18:56:18.098 34 INFO nova.metadata....
3,2023-05-10 18:56:18.117 35 INFO nova.metadata....
4,2023-05-10 18:56:18.120 13 INFO nova.osapi_com...


In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/thesis/val/raw_val.csv', sep=',')

In [ ]:
df2.shape

(102000, 1)

In [ ]:
df2.rename(columns={'message':'Content'}, inplace=True)

In [ ]:
#df2.Content[268025]

In [ ]:
#df2.Content[297871]

In [ ]:
df2['Content']=df2['Content'].str.replace(r"len.*", "", regex=True)

In [ ]:
df2.head

<bound method NDFrame.head of                                                   Content
0       2023-05-10 18:56:18.091 33 INFO nova.metadata....
1       2023-05-10 18:56:18.094 34 INFO nova.metadata....
2       2023-05-10 18:56:18.098 34 INFO nova.metadata....
3       2023-05-10 18:56:18.117 35 INFO nova.metadata....
4       2023-05-10 18:56:18.120 13 INFO nova.osapi_com...
...                                                   ...
101995                                 warnings.warn(msg)
101996  /usr/lib/python3/dist-packages/oslo_policy/pol...
101997                                 warnings.warn(msg)
101998                                 warnings.warn(msg)
101999  /usr/lib/python3/dist-packages/oslo_policy/pol...

[102000 rows x 1 columns]>

In [ ]:
df2['Content']=df2['Content'].str.replace(r'^([\d-]*) ([\d\.:]*) (\d+)', '', regex=True) # Remove date
df2['Content']=df2['Content'].str.replace(r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', '', regex=True) #ip address
df2['Content']=df2['Content'].str.replace(r"\b\w{8}-\w{4}-\w{4}-\w{4}-\w{12}\b", '', regex=True) #remove server ids
df2['Content']=df2['Content'].str.replace(r'add', '', regex=True) # remove add from policy
df2['Content']=df2['Content'].str.replace(r'xbh', '', regex=True) # remove the xbh

In [ ]:
status_code_pattern = r'\b(?:0x|#)?([1-5]\d{2}|4\d{1}[1-9]|4[1-4]\d|40[1-9]|5[0-5]\d|500)\b'
# Extract status codes from sentences and add to new column
df2['Status_Code'] = df2['Content'].apply(lambda x: re.findall(status_code_pattern, x))

In [ ]:
df2.head(5)

,Content,Status_Code
0,"INFO nova.metadata.wsgi.server [-] ,,, ""GET /...",[200]
1,"INFO nova.metadata.wsgi.server [-] ,,, ""GET /...",[200]
2,"INFO nova.metadata.wsgi.server [-] ,,, ""GET /...",[200]
3,"INFO nova.metadata.wsgi.server [-] ,,, ""GET /...",[200]
4,INFO nova.osapi_compute.wsgi.server [req- 507...,[200]


In [ ]:
df2['Content']=df2['Content'].str.replace(r'\b\d{4,}\b', '', regex=True) #remove numer that are not http status codes
df2['Content']=df2['Content'].str.replace(r'\/.*\/', '', regex=True) #remove the path
df2['Content']=df2['Content'].str.replace(r'\[.*\]', '', regex=True) #remove everything inside square brackets usually req ids
df2['Content']=df2['Content'].str.replace(r'[0-9a-fA-F]{40}', ' ', regex=True) #hexadecimal character removal
df2['Content']=df2['Content'].str.replace(r'\b\w{1}\b','',regex=True) #remove single letter
df2['Content']=df2['Content'].str.replace(r'\d', " ", regex=True) # remove any diigit with global search
df2['Content']=df2['Content'].str.replace(r'/\W/', '', regex=True) #remove any non word character
df2['Content']=df2['Content'].str.replace(r'\W', " ", regex=True) #anything but a number, letter and _
df2['Content']=df2['Content'].str.replace(r'\s+',' ',regex=True) # remove extra spaces

In [ ]:
df2.head(5)

,Content,Status_Code
0,INFO nova metadata wsgi server GET status,[200]
1,INFO nova metadata wsgi server GET status,[200]
2,INFO nova metadata wsgi server GET status,[200]
3,INFO nova metadata wsgi server GET status,[200]
4,INFO nova osapi_compute wsgi server GET status,[200]


In [ ]:
df2.shape

(102000, 2)

In [ ]:
#df2.Content[49823]

In [ ]:
df2.tail(10)

,Content,Status_Code
101990,policy py UserWarning Policy os_compute_api se...,[]
101991,policy py UserWarning Policy os_compute_api se...,[]
101992,warnings warn msg,[]
101993,policy py UserWarning Policy os_compute_api se...,[]
101994,policy py UserWarning Policy os_compute_api se...,[]
101995,warnings warn msg,[]
101996,policy py UserWarning Policy os_compute_api se...,[]
101997,warnings warn msg,[]
101998,warnings warn msg,[]
101999,policy py UserWarning Policy os_compute_api se...,[]


In [ ]:
df2.Content[163]

' INFO nova osapi_compute wsgi server GET status '

In [ ]:
df2['Status_Code']=df2['Status_Code'].astype(str)

In [ ]:
df2['Status_Code']=df2['Status_Code'].str.replace(r'\W', " ", regex=True) #anything but a number, letter and _

In [ ]:
df2['Content']=df2['Content'].str.lower()

In [ ]:
df2['clean_content']=df2['Content']+ '' +df2['Status_Code']

In [ ]:
df2['clean_content'].head(5)

0     info nova metadata wsgi server get status   2...
1     info nova metadata wsgi server get status   2...
2     info nova metadata wsgi server get status   2...
3     info nova metadata wsgi server get status   2...
4     info nova osapi_compute wsgi server get statu...
Name: clean_content, dtype: object

In [ ]:
pattern = r'xbh'

In [ ]:
rows_mtch = df2[df2['clean_content'].str.contains(pattern, regex=True)]

In [ ]:
print(rows_mtch)

Empty DataFrame
Columns: [Content, Status_Code, clean_content]
Index: []


In [ ]:
#df2.to_csv('/content/drive/MyDrive/thesis/report-wordtokenize-novapi.csv', index=False)
# use df2.to_csv('/content/drive/MyDrive/thesis/models/lof/report-wordtoken-outlier-novapi.csv', index=False)
df2.to_csv('/content/drive/MyDrive/thesis/val/val-wordtoken-novapi.csv', index=False)
#df2.to_csv('/content/drive/MyDrive/thesis/train/wordtoken-novapi.csv', index=False)

In [ ]:
#Remove the stopwords from the nltk stopword list
nltk.download('stopwords')
import nltk
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df2['content'] = df2['clean_content'].apply(lambda x: word_tokenize(x))

In [ ]:
df2.to_csv('/content/drive/MyDrive/thesis/val/val-wordtokenize-novapiv2.csv', index=False)
#df2.to_csv('/content/drive/MyDrive/thesis/wordtokenize-novapiv2.csv', index=False)

In [ ]:
df2.head(5)

,Content,Status_Code,clean_content,content
0,info nova metadata wsgi server get status,200,info nova metadata wsgi server get status 2...,"[info, nova, metadata, wsgi, server, get, stat..."
1,info nova metadata wsgi server get status,200,info nova metadata wsgi server get status 2...,"[info, nova, metadata, wsgi, server, get, stat..."
2,info nova metadata wsgi server get status,200,info nova metadata wsgi server get status 2...,"[info, nova, metadata, wsgi, server, get, stat..."
3,info nova metadata wsgi server get status,200,info nova metadata wsgi server get status 2...,"[info, nova, metadata, wsgi, server, get, stat..."
4,info nova osapi_compute wsgi server get status,200,info nova osapi_compute wsgi server get statu...,"[info, nova, osapi_compute, wsgi, server, get,..."
